In [1]:
import torch
import data as Data
import model as Model
import argparse
import logging
import core.logger as Logger
import core.metrics as Metrics
from tensorboardX import SummaryWriter
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
class args_standin():
    def __init__(self, config, phase, gpu_ids):
        self.config = config
        self.phase = phase
        self.gpu_ids = gpu_ids
        self.debug = False

# args = args_standin('config/mri_tibia.json', 'val', None)
args = args_standin('config/mri_condensed_femur.json', 'val', None)


opt = Logger.parse(args)

export CUDA_VISIBLE_DEVICES=0


In [3]:
# opt['path']['resume_state'] = 'experiments/sr_ffhq_211103_145403/checkpoint/I160000_E1034'
# opt['datasets']['val']['dataroot'] = 'data/datasets/tibia/tester_inputs'

opt['path']['resume_state'] = 'experiments/sr_ffhq_211108_120031/checkpoint/I110000_E330'
opt['datasets']['val']['dataroot'] = 'data/datasets/condensed_femur/val_96_288'

In [4]:
val_set = Data.create_dataset(opt['datasets']['val'], 'val')
val_loader = Data.create_dataloader(
        val_set, opt['datasets']['val'], 'val')

In [5]:
diffusion = Model.create_model(opt)

In [6]:
diffusion.set_new_noise_schedule(opt['model']['beta_schedule'][opt['phase']], schedule_phase=opt['phase'])

In [7]:
def test_image(val_data, model, result_path, scalefactor):
    diffusion=model
    diffusion.feed_data(val_data)
    diffusion.test(continous=False)
    visuals = diffusion.get_current_visuals()
    sr_img = Metrics.tensor2img(visuals['SR'])  # uint8
    hr_img = Metrics.tensor2img(visuals['HR'])  # uint8
    lr_img = Metrics.tensor2img(visuals['LR'])  # uint8
    fake_img = Metrics.tensor2img(visuals['INF'])  # uint8
    lr_img = np.repeat(np.repeat(lr_img, scalefactor, axis=0), scalefactor, axis=1)
    out_img = np.concatenate((hr_img,lr_img,fake_img,sr_img),axis=1)
    Metrics.save_img(out_img, '{}.png'.format(result_path))

In [8]:
def test_image_continuous(val_data, model, result_path, scalefactor):
    diffusion=model
    diffusion.feed_data(val_data)
    diffusion.test(continous=True)
    visuals = diffusion.get_current_visuals()
    sr_img = Metrics.tensor2img(visuals['SR'])  # uint8
    srflat = np.concatenate([sr_img[:,:,i] for i in range(len(sr_img[0,0,:]))],axis=1)
    Metrics.save_img(srflat, '{}.png'.format(result_path))

In [11]:
idx = [6,21]#[1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
scalefactor = int(opt['datasets']['train']['r_resolution']/opt['datasets']['train']['l_resolution'])

for i,  val_data in enumerate(val_loader):
    if i in idx:
        test_image_continuous(val_data, diffusion, "misc/continuousim_{}".format(i), scalefactor)
    elif i > max(idx):
        break
    else:
        continue

sampling loop time step: 100%|██████████| 2000/2000 [02:29<00:00, 13.42it/s]


In [11]:
#idx = [500, 1000, 1500, 2000, 2500, 3000]
idx = [17]
scalefactor = int(opt['datasets']['train']['r_resolution']/opt['datasets']['train']['l_resolution'])

for i,  val_data in enumerate(val_loader):
    if i in idx:
        test_image(val_data, diffusion, "misc/testim_128-256_{}_mri_r0".format(i), scalefactor)
    elif i > max(idx): 
        break
    else:
        continue
#     val_data = np.transpose(np.tile(np.squeeze(np.array(val_data['HR'])),(3,1,1)),[1,2,0])
#     val_data = -1*val_data
#     plt.imshow(val_data)
    

sampling loop time step: 100%|██████████| 2000/2000 [03:07<00:00, 10.67it/s]


In [ ]:
idx = 0
current_epoch = opt['path']['resume_state'][-4:]
avg_psnr = 0
for _,  val_data in enumerate(val_loader):
    idx += 1
    diffusion.feed_data(val_data)
    diffusion.test(continous=False)
    visuals = diffusion.get_current_visuals()
    sr_img = Metrics.tensor2img(visuals['SR'])  # uint8
    hr_img = Metrics.tensor2img(visuals['HR'])  # uint8
    lr_img = Metrics.tensor2img(visuals['LR'])  # uint8
    fake_img = Metrics.tensor2img(visuals['INF'])  # uint8

    # generation
    Metrics.save_img(
        hr_img, 'misc/tibia_reps/{}_{}_hr.png'.format(current_epoch, idx))
    Metrics.save_img(
        sr_img, 'misc/tibia_reps/{}_{}_sr.png'.format(current_epoch, idx))
    Metrics.save_img(
        lr_img, 'misc/tibia_reps/{}_{}_lr.png'.format(current_epoch, idx))
    Metrics.save_img(
        fake_img, 'misc/tibia_reps/{}_{}_inf.png'.format(current_epoch, idx))
    if len(sr_img.shape) == 3:# this should only activate if the image is 3 channel, if one channel image, no need to transpose
        tb_logger.add_image(
            'Iter_{}'.format(current_step),
            np.transpose(np.concatenate(
                (fake_img, sr_img, hr_img), axis=1), [2, 0, 1]),
            idx)
    else:
        Metrics.save_img(np.concatenate(
                (fake_img, hr_img, sr_img), axis=1),
            'misc/{}_{}.png'.format(current_epoch, idx))

    avg_psnr += Metrics.calculate_psnr(
        sr_img, hr_img)
#     if idx == 3:
#         break

avg_psnr = avg_psnr / idx
print('{}_{}'.format(current_epoch, avg_psnr))

sampling loop time step:  77%|███████▋  | 1537/2000 [01:50<00:33, 13.73it/s]

In [ ]:
2
3
(4)
6
7
9
12
(14)
(15)
17
(18)
22
24
25


In [11]:
ixs = [25]
idx = 0
current_epoch = opt['path']['resume_state'][-4:]
avg_psnr = 0
# for j in range(1)
for _,  val_data in enumerate(val_loader):
    idx += 1
    if idx not in ixs:
        continue
    diffusion.feed_data(val_data)
    diffusion.test(continous=False)
    visuals = diffusion.get_current_visuals()
    sr_img = Metrics.tensor2img(visuals['SR'])  # uint8
    hr_img = Metrics.tensor2img(visuals['HR'])  # uint8
    lr_img = Metrics.tensor2img(visuals['LR'])  # uint8
    fake_img = Metrics.tensor2img(visuals['INF'])  # uint8

    # generation
    Metrics.save_img(-
        hr_img, 'misc/condensed_femur_reps/{}_{}_hr.png'.format(current_epoch, idx))
    Metrics.save_img(
        sr_img, 'misc/condensed_femur_reps/{}_{}_sr.png'.format(current_epoch, idx))
    Metrics.save_img(
        lr_img, 'misc/condensed_femur_reps/{}_{}_lr.png'.format(current_epoch, idx))
    Metrics.save_img(
        fake_img, 'misc/condensed_femur_reps/{}_{}_inf.png'.format(current_epoch, idx))
    if len(sr_img.shape) == 3:# this should only activate if the image is 3 channel, if one channel image, no need to transpose
        tb_logger.add_image(
            'Iter_{}'.format(current_step),
            np.transpose(np.concatenate(
                (fake_img, sr_img, hr_img), axis=1), [2, 0, 1]),
            idx)
    else:
        Metrics.save_img(np.concatenate(
                (fake_img, hr_img, sr_img), axis=1),
            'misc/{}_{}.png'.format(current_epoch, idx))

    avg_psnr += Metrics.calculate_psnr(
        sr_img, hr_img)
#     if idx == 3:
#         break

avg_psnr = avg_psnr / idx
print('{}_{}'.format(current_epoch, avg_psnr))

sampling loop time step: 100%|██████████| 2000/2000 [03:06<00:00, 10.73it/s]


_E60_0.1664998063404325
